In [1]:
import os
import sys
import json
import ndex2
from ndex2.nice_cx_network import NiceCXNetwork
from ndex2.nice_cx_network import DefaultNetworkXFactory
import networkx as nx
import requests

In [2]:
network =  ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='38a9cebf-23dc-11e8-b939-0ac135e8bacf') # (Nodes1127)

In [3]:
tmp=set()
for e in network.get_edges():
    tmp.add(e[1]['s'])
    tmp.add(e[1]['t'])
tmp_dict={}
for i,j in enumerate(tmp):
    tmp_dict.update({j:i})

In [4]:
edge_data=""
for e in network.get_edges():
    edge_data+=str(tmp_dict[e[1]['s']])+"\t"+str(tmp_dict[e[1]['t']])+"\n"

In [5]:
template={
          "algorithm": "louvain",
          "graphdirected": "false",
          "data": "",
          "ipAddress": "string",
          "customParameters": {
            "additionalProp1": "string",
            "additionalProp2": "string",
            "additionalProp3": "string"
          }
        }

In [6]:
template['data']=edge_data

In [7]:
res = requests.post('http://ddot.ucsd.edu/cd/communitydetection/v1',json=template)
print('Created task. ID: {}'.format(res.json()["id"]))

Created task. ID: 9218f2c7-ba27-4bf8-942f-c13707feb1bd


In [13]:
res = requests.get("http://ddot.ucsd.edu/cd/communitydetection/v1/"+res.json()['id'])
res.json()['status']

'complete'

In [14]:
if(res.json()['status']!='complete'):
    print("*********    Calculating!!!!    Wait!!!!!     *********\n\n\n")

In [15]:
class UnionFind:
    def __init__(self, n):
        # negative : root
        # non-negative : rank
        self.table = [-1] * n
 
    def _root(self, x):
        if self.table[x] < 0:
            #print(x)
            return x
        else:
            self.table[x] = self._root(self.table[x])
            return self.table[x]
 
    def find(self, x, y):
        return self._root(x) == self._root(y)
 
    def unite(self, x, y):
        r1 = self._root(x)
        r2 = self._root(y)
        if r1 == r2:
            return
        d1 = self.table[r1]
        d2 = self.table[r2]
        if d1 <= d2:
            self.table[r2] = r1
            if d1 == d2:
                self.table[r1] -= 1
        else:
            self.table[r1] = r2


In [18]:
def communities_from_clusterfile(data,rank):
    hierarchy = 0
    clist = []
    flag = True
    node_list=set()
    UF=UnionFind(int(data.split(';')[-2].split(',')[0]))
    for line in reversed(data.split(';')):
        slist = line.split(',')
        if len(slist) != 3:
            continue
        if slist[0] in clist:
            hierarchy += 1
            clist.clear()
        else:
            clist.append(slist[1])
        if slist[2].startswith('c-m'):
            node_list.add(int(slist[1]))
        
        if hierarchy >= rank:
            UF.unite(int(slist[0]),int(slist[1]))
    if hierarchy < rank:
        print("Your rank is larger than hierarchy\n")
        sys.exit()
    print("max rank = ", hierarchy)
    return UF.table[0:len(node_list)]

In [19]:
communities=communities_from_clusterfile(res.json()['result'],1)

max rank =  1


In [21]:
import sys
import json
import ndex2
from auto_graph_visualizer.utils import *
from auto_graph_visualizer.visualizer import AutoGraphVisualizer

In [22]:
options = {
        "graph_name": "test_network",
        "output_file_name": "test_out",
        "algorithm": "rest",
        "color_palette": "hls",
        "nodesize": "betweenness",
        "maxnodesize": 1000000,
        "density": "normal",
        "positions": "fa",
    }



In [23]:
viz = AutoGraphVisualizer(options,res.json()['result'])

In [24]:
cx_network = network.to_cx() 
    
cxobj = viz.generate_viz(cx_network)

  0%|          | 0/2000 [00:00<?, ?it/s]

Generating CX
Name: BioGRID: Protein-Protein Interactions (HIV-1)
Nodes: 1136
Edges: 1324
Node Attributes: 3323
Edge Attributes: 9268



100%|██████████| 2000/2000 [00:27<00:00, 73.19it/s] 


BarnesHut Approximation  took  9.09  seconds
Repulsion forces  took  15.28  seconds
Gravitational forces  took  0.17  seconds
Attraction forces  took  0.01  seconds
AdjustSpeedAndApplyForces step  took  0.98  seconds
Generating CX


In [25]:
from py2cytoscape import cyrest

# This should be changed to your host IP
host_ip = '127.0.0.1'
cytoscape=cyrest.cyclient(host=host_ip)
cytoscape.version()
cytoscape.status()

apiVersion v1
cytoscapeVersion 3.7.2
CyREST online!


In [26]:
cytoscape.session.new()

In [27]:
import requests

cyrest_url = 'http://' + host_ip + ':1234/cyndex2/v1/networks/cx'
print(cyrest_url)
response = requests.post(cyrest_url, data=json.dumps(cxobj))
print(response.json())

http://127.0.0.1:1234/cyndex2/v1/networks/cx
{'data': {'suid': 52, 'uuid': ''}, 'errors': []}
